In [22]:
from pycox.evaluation import EvalSurv
import torch
import torchtuples as tt
import os
import pickle
from lifelines.utils import concordance_index

from utils import dataset_name
from eval_utils import MODEL_CLASS, get_results, DL_MODELS, ML_MODELS, MODELS
import numpy as np
import pandas as pd
np.random.seed(42)
_ = torch.manual_seed(42)


print(f"Experiments over {dataset_name}")

Experiments over hepatoCellular


In [23]:
train = pd.read_csv(f"../../datasets/train/{dataset_name}.csv")
test = pd.read_csv(f"../../datasets/test/{dataset_name}.csv")

X, y = train.drop(["event", "duration"], axis=1), train[["event", "duration"]]
X_test, y_test = test.drop(["event", "duration"], axis=1), test[["event", "duration"]]

# Finding the best parameters for each model and run the models over the test data

In [24]:
results = pd.DataFrame()
     
for model in MODELS:
    stats = pickle.load( open( f"statistics/{model}/best_model.pkl", "rb" ))
    df = pd.DataFrame(stats, index=[model])
    results = pd.concat([results, df], axis=0)
    
c_index_df = results[["c_index", "c_index_std", "c_index_params"]].sort_values("c_index", ascending=False)
concordance_td_df = results[["concordance_td", "concordance_td_std", "concordance_td_params"]].sort_values("concordance_td", ascending=False)
ibs_df = results[["ibs", "ibs_std", "ibs_params"]].sort_values("ibs", ascending=True)

## `c index`

In [25]:
c_index_df

,c_index,c_index_std,c_index_params
cox_time,0.725330,0.047206,"cox_time_dropout_0.1_num_nodes_[28, 28, 100, 2..."
deep_surv,0.719306,0.069547,"deep_surv_dropout_0.2_num_nodes_[16, 32, 64, 6..."
gbst,0.696838,0.065022,gbst_learning_rate_0.1_n_estimators_100_subsam...
rsf,0.679879,0.025176,rsf_n_estimators_200_max_depth_10_min_samples_...
reg_coxph,0.619029,0.053475,reg_coxph_l1_ratio_0.1_tol_0.1_max_iter_1000_v...
deep_hit,0.603726,0.037770,"deep_hit_dropout_0.3_num_nodes_[3, 5]_activati..."
pc_hazard,0.594232,0.048275,"pc_hazard_dropout_0.1_num_nodes_[256, 256]_act..."


In [26]:
results = []
column = "c_index_params"


for model_name, params in zip(c_index_df.index, c_index_df[column]):
    if model_name in DL_MODELS:
        c_index, concordance_td, ibs = get_results(params, model_name, X, y, X_test, y_test)
        results.append((model_name, c_index))
    else:
        c_index, concordance_td, ibs = get_results(params, model_name, X, y, X_test, y_test)
        results.append((model_name, c_index))

for m,val in sorted(results, key=lambda x: x[1], reverse=True):
    print(m, np.round(val,4))

cox_time 0.8125
gbst 0.8
rsf 0.7938
deep_surv 0.7562
reg_coxph 0.7375
pc_hazard 0.575
deep_hit 0.3625


___

## `concordance td`

In [27]:
concordance_td_df

,concordance_td,concordance_td_std,concordance_td_params
deep_surv,0.719306,0.069547,"deep_surv_dropout_0.2_num_nodes_[16, 32, 64, 6..."
cox_time,0.708213,0.045099,"cox_time_dropout_0.1_num_nodes_[28, 28, 100, 2..."
deep_hit,0.702880,0.091373,"deep_hit_dropout_0.3_num_nodes_[28, 28, 100, 2..."
gbst,0.696838,0.065022,gbst_learning_rate_0.1_n_estimators_100_subsam...
rsf,0.686449,0.014973,rsf_n_estimators_100_max_depth_10_min_samples_...
reg_coxph,0.619029,0.053475,reg_coxph_l1_ratio_0.1_tol_0.1_max_iter_1000_v...
pc_hazard,0.077157,0.069519,"pc_hazard_dropout_0.3_num_nodes_[256, 256]_act..."


In [28]:
results = []
column = "concordance_td_params"


for model_name, params in zip(concordance_td_df.index, concordance_td_df[column]):
    if model_name in DL_MODELS:
        c_index, concordance_td, ibs = get_results(params, model_name, X, y, X_test, y_test)
        results.append((model_name, concordance_td))
    else:
        c_index, concordance_td, ibs = get_results(params, model_name, X, y, X_test, y_test)
        results.append((model_name, concordance_td))

for m,val in sorted(results, key=lambda x: x[1], reverse=True):
    print(m, np.round(val,4))

cox_time 0.8312
gbst 0.8
rsf 0.7938
deep_hit 0.7625
deep_surv 0.7562
reg_coxph 0.7375
pc_hazard 0.6062


## `integrated brier score`

In [29]:
ibs_df

,ibs,ibs_std,ibs_params
pc_hazard,0.050323,0.016709,"pc_hazard_dropout_0.3_num_nodes_[256, 256]_act..."
deep_hit,0.159163,0.038461,"deep_hit_dropout_0.1_num_nodes_[32, 32]_activa..."
deep_surv,0.169705,0.035928,"deep_surv_dropout_0.2_num_nodes_[16, 32, 64, 6..."
cox_time,0.172812,0.021892,"cox_time_dropout_0.1_num_nodes_[28, 28, 100, 2..."
rsf,0.182383,0.009547,rsf_n_estimators_100_max_depth_10_min_samples_...
gbst,0.187758,0.017340,gbst_learning_rate_0.1_n_estimators_50_subsamp...
reg_coxph,0.216893,0.013891,reg_coxph_l1_ratio_0.1_tol_0.1_max_iter_1000_v...


In [30]:
results = []
column = "ibs_params"


for model_name, params in zip(ibs_df.index, ibs_df[column]):
    if model_name in DL_MODELS:
        c_index, concordance_td, ibs = get_results(params, model_name, X, y, X_test, y_test)
        results.append((model_name, ibs))
    else:
        c_index, concordance_td, ibs = get_results(params, model_name, X, y, X_test, y_test)
        results.append((model_name, ibs))

for m,val in sorted(results, key=lambda x: x[1], reverse=False):
    print(m, np.round(val,4))

deep_hit 0.1497
deep_surv 0.1535
gbst 0.1574
cox_time 0.1597
rsf 0.1605
reg_coxph 0.195
pc_hazard 0.5067
